# ADMET Group classification tasks
This notebook contains the benchmark of distilled models of Zaira-Chem with the Olinda pipeline for the ADMET datasets from Therapeutics Data Commons with classification tasks

In [1]:
import os
import pandas as pd
import numpy as np

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

DATAPATH = "../data"
PREDPATH = "../predictions"

Found local copy...


In [2]:
admet_datasets = ["Bioavailability_Ma",
                  "HIA_Hou", 
                  "Pgp_Broccatelli",
                  "BBB_Martins",
                  "CYP2C9_Veith", 
                  "CYP2D6_Veith", 
                  "CYP3A4_Veith",
                  "CYP2C9_Substrate_CarbonMangels", 
                  "CYP2D6_Substrate_CarbonMangels",
                  "CYP3A4_Substrate_CarbonMangels",    
                  "hERG", 
                  'AMES', 
                  "DILI",
                ]

## Data preparation

In [3]:
#save train and test sets cleaned for ZairaChem inputs
for a in admet_datasets:
    benchmark = group.get(a)
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train_val.drop(columns=["Drug_ID"], inplace=True)
    test.drop(columns=["Drug_ID"], inplace=True)
    test.to_csv(os.path.join(DATAPATH, "{}_test.csv".format(a)), index=False)
    train_val.to_csv(os.path.join(DATAPATH, "{}_train.csv".format(a)), index=False)

## Model Training
Files in the /data folder can be directly used as ZairaChem inputs. A minimum of 5 folds per assay is required. 

ZairaChem performs its own train/validation splits automatically, so the whole train_val set must be passed as input for model training.

```
zairachem fit -i <dataset>_train.csv -m <dataset>_fold1
zairachem predict -i <dataset>_test.csv -m <dataset>_fold1 -o <dataset>_pred_fold1
zairachem distill -m <dataset>_fold1 -o <dataset>_fold1.onnx
```

## Model Evaluation

### ZairaChem
First we re-train the ZairaChem models as some of the default descriptors have changed (TabPFN no longer used). 

As an example, we provide the results of 5 folds of ZairaChem models for each dataset. The automated reports generated by ZairaChem, as well as raw data outputs is shared. 

An example model is also provided in the /model folder.

*Some molecules cannot be inferred using the ZairaChem pipeline due to the molecular descriptors used. The results for these molecules is calculated as the mean of all the predictions, and the molecules are saved for manual inspection*

In [4]:
max_fold = 5

In [5]:
zairachem_predictions_list = []
nan_mols = {}

for i in range(1,max_fold+1):
    predictions = {}
    for a in admet_datasets:
        path = os.path.join("../predictions/tdc/zaira-chem", a)
        pred_file = pd.read_csv(os.path.join(path, "fold{}".format(i), "report", "output_table.csv"))
        y_pred_test = pred_file["pred-value"]
        if i == 1: #keep molecules for which preds are not calculated once
            smi = []
            for n,y in enumerate(y_pred_test):
                if np.isnan(y):
                    smi += [pred_file.loc[n]["input-smiles"]]
            nan_mols[a] = smi
        #replace Nan values by mean values
        arr = np.array(y_pred_test)
        mean_val = np.nanmean(arr)
        arr[np.isnan(arr)] = mean_val
        y_pred_test = arr.tolist()
        predictions[a]= y_pred_test
        
    zairachem_predictions_list.append(predictions)        

In [6]:
zairachem_results = group.evaluate_many(zairachem_predictions_list)
zairachem_results

{'bioavailability_ma': [0.693, 0.009],
 'hia_hou': [0.99, 0.004],
 'pgp_broccatelli': [0.941, 0.005],
 'bbb_martins': [0.921, 0.007],
 'cyp2c9_veith': [0.779, 0.005],
 'cyp2d6_veith': [0.707, 0.01],
 'cyp3a4_veith': [0.858, 0.007],
 'cyp2c9_substrate_carbonmangels': [0.42, 0.011],
 'cyp2d6_substrate_carbonmangels': [0.719, 0.023],
 'cyp3a4_substrate_carbonmangels': [0.633, 0.01],
 'herg': [0.861, 0.005],
 'ames': [0.853, 0.005],
 'dili': [0.933, 0.012]}

### Olinda
The corresponding distilled ONNX models from Olinda for each of the ZairaChem models above.

The goal is to quanitfy the performance retained by the Olinda models from the original ZairaChem model.

We use a wrapper api https://github.com/JHlozek/olinda_api to simplify the model usage.

In [ ]:
import onnx_runner
olinda_predictions_list = []
nan_mols = {}

for i in range(1,max_fold+1):
    predictions = {}
    for a in admet_datasets:
        path = os.path.join("../models/tdc_models") #, a)      
        onnx_model = onnx_runner.onnx_runner(os.path.join(path, a, a + "_fold{}".format(i) + ".onnx"))
        test_df = pd.read_csv(os.path.join(DATAPATH, "{}_test.csv".format(a)))
        test_smiles_list = test_df["Drug"].tolist()
        
        y_pred_test = onnx_model.predict(test_smiles_list)

        if i == 1: #keep molecules for which preds are not calculated once
            smi = []
            for n,y in enumerate(y_pred_test):
                if np.isnan(y):
                    smi += [test_smiles_list[n]]
            nan_mols[a] = smi
        #replace Nan values by mean values
        arr = np.array(y_pred_test)
        mean_val = np.nanmean(arr)
        arr[np.isnan(arr)] = mean_val
        y_pred_test = arr.tolist()
        
        predictions[a]= y_pred_test
    olinda_predictions_list.append(predictions)        

In [8]:
olinda_results = group.evaluate_many(olinda_predictions_list)
olinda_results

{'bioavailability_ma': [0.676, 0.022],
 'hia_hou': [0.888, 0.194],
 'pgp_broccatelli': [0.936, 0.006],
 'bbb_martins': [0.899, 0.009],
 'cyp2c9_veith': [0.744, 0.006],
 'cyp2d6_veith': [0.652, 0.012],
 'cyp3a4_veith': [0.846, 0.003],
 'cyp2c9_substrate_carbonmangels': [0.415, 0.019],
 'cyp2d6_substrate_carbonmangels': [0.693, 0.022],
 'cyp3a4_substrate_carbonmangels': [0.615, 0.01],
 'herg': [0.843, 0.021],
 'ames': [0.823, 0.005],
 'dili': [0.914, 0.023]}

In [ ]:
comparison = []
for i in olinda_results:
    comparison.append(olinda_results[i][0] / zairachem_results[i][0])
print(round(sum(comparison)/len(comparison), 3))

### Olinda original binary data

Lastly, we want to check if we should simply use the underlying Morgan FP and KerasTuner paradigm to train a model directly on the original binarized experimental data instead of first trainging a ZairaChem model and the training a surrogate with Olinda.

In [ ]:
import onnx_runner
olinda_direct_predictions_list = []
nan_mols = {}

for i in range(1,max_fold+1):
    predictions = {}
    for a in admet_datasets:
        path = os.path.join("../models/tdc_direct_true") #, a)      
        onnx_model = onnx_runner.onnx_runner(os.path.join(path, a, a + "_direct_fold{}".format(i) + ".onnx"))
        test_df = pd.read_csv(os.path.join(DATAPATH, "{}_test.csv".format(a)))
        test_smiles_list = test_df["Drug"].tolist()
        
        y_pred_test = onnx_model.predict(test_smiles_list)

        if i == 1: #keep molecules for which preds are not calculated once
            smi = []
            for n,y in enumerate(y_pred_test):
                if np.isnan(y):
                    smi += [test_smiles_list[n]]
            nan_mols[a] = smi
        #replace Nan values by mean values
        arr = np.array(y_pred_test)
        mean_val = np.nanmean(arr)
        arr[np.isnan(arr)] = mean_val
        y_pred_test = arr.tolist()
        
        predictions[a]= y_pred_test
    olinda_direct_predictions_list.append(predictions)        

In [10]:
olinda_direct_results = group.evaluate_many(olinda_direct_predictions_list)
olinda_direct_results

{'bioavailability_ma': [0.617, 0.053],
 'hia_hou': [0.859, 0.045],
 'pgp_broccatelli': [0.924, 0.005],
 'bbb_martins': [0.786, 0.018],
 'cyp2c9_veith': [0.717, 0.01],
 'cyp2d6_veith': [0.609, 0.026],
 'cyp3a4_veith': [0.823, 0.01],
 'cyp2c9_substrate_carbonmangels': [0.387, 0.021],
 'cyp2d6_substrate_carbonmangels': [0.654, 0.059],
 'cyp3a4_substrate_carbonmangels': [0.586, 0.028],
 'herg': [0.767, 0.033],
 'ames': [0.788, 0.011],
 'dili': [0.807, 0.026]}

### Combine all results

In [13]:
zaira_rocs = [val[0] for val in zairachem_results.values()]
zaira_stdev = [val[1] for val in zairachem_results.values()]
olinda_rocs = [val[0] for val in olinda_results.values()]
olinda_stdev = [val[1] for val in olinda_results.values()]
olinda_direct_rocs = [val[0] for val in olinda_direct_results.values()]
olinda_direct_stdev = [val[1] for val in olinda_direct_results.values()]

diffs = []
fraction = []
for i in range(len(zaira_rocs)):
    tmp = round(zaira_rocs[i] - olinda_rocs[i], 3)
    if tmp > 0:
        diffs.append("-" + str(abs(tmp)))
    else:
        diffs.append("+" + str(abs(tmp)))

    fraction.append(round(olinda_rocs[i] / zaira_rocs[i], 2))

df = pd.DataFrame(columns=["assay", "zaira-chem auroc", "zaira-chem stdev", "olinda auroc", "olinda stdev", "delta", "perc_performance", "olinda direct auroc", "olinda direct stdev"], data=list(zip(admet_datasets, zaira_rocs, zaira_stdev, olinda_rocs, olinda_stdev, diffs, fraction, olinda_direct_rocs, olinda_direct_stdev)))

In [14]:
df

,assay,zaira-chem auroc,zaira-chem stdev,olinda auroc,olinda stdev,delta,perc_performance,olinda direct auroc,olinda direct stdev
0,Bioavailability_Ma,0.693,0.009,0.676,0.022,-0.017,0.98,0.617,0.053
1,HIA_Hou,0.990,0.004,0.888,0.194,-0.102,0.90,0.859,0.045
2,Pgp_Broccatelli,0.941,0.005,0.936,0.006,-0.005,0.99,0.924,0.005
3,BBB_Martins,0.921,0.007,0.899,0.009,-0.022,0.98,0.786,0.018
4,CYP2C9_Veith,0.779,0.005,0.744,0.006,-0.035,0.96,0.717,0.010
5,CYP2D6_Veith,0.707,0.010,0.652,0.012,-0.055,0.92,0.609,0.026
6,CYP3A4_Veith,0.858,0.007,0.846,0.003,-0.012,0.99,0.823,0.010
7,CYP2C9_Substrate_CarbonMangels,0.420,0.011,0.415,0.019,-0.005,0.99,0.387,0.021
8,CYP2D6_Substrate_CarbonMangels,0.719,0.023,0.693,0.022,-0.026,0.96,0.654,0.059
9,CYP3A4_Substrate_CarbonMangels,0.633,0.010,0.615,0.010,-0.018,0.97,0.586,0.028
